In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
from ClusterAndPredict.ClusterAndPredict import ClusterAndPredict
from Testing.DataLoader import DataLoader
from Testing.ParameterCreator import ParameterCreator

In [2]:
# DEPRECATED
# chroma_client = chromadb.PersistentClient(path="./../../Clustering/Clustering/Chroma")
# # Count number of collections
# print(chroma_client.count_collections())
# 
# # Get all collection names
# collection_names = chroma_client.list_collections()
# 
# # Loop through each collection and drop it
# for collection_name in collection_names:
#     if collection_name.name != 'climate_claims_embeddings_unchanged':
#         chroma_client.delete_collection(collection_name.name)

In [3]:
params = ParameterCreator().get_parameters()
results = []
cluster_dfs = []
for param in params:
    percentage = 1
    data_loader = DataLoader(percentage, True, param['random_seed'])
    train_df, test_df = data_loader.create_train_test_df(True, True, True)
    clf = ClusterAndPredict(**param, train_df=train_df)
    clf.fit(test_df['Text'].tolist(), test_df['Numerical Rating'].tolist())
    # Print best parameters
    best_estimator = clf
    score = best_estimator.score([], [])
    print(best_estimator.get_all_performance_metrics())
    object_output = best_estimator.get_all_performance_metrics()
    cluster_df = object_output['cluster_df']
    cluster_dfs.append(cluster_df)
    output = {
        'percentage': percentage,
        'score': score,
        'accuracy': best_estimator.get_accuracy(),
        'was_supervised_umap_used': best_estimator.get_was_supervised(),
        'metrics': best_estimator.get_all_performance_metrics(),
    }
    print(output)
    results.append(output)

Numerical Rating
1    2990
3     619
Name: count, dtype: int64
Numerical Rating
1    778
3    124
Name: count, dtype: int64
cluster  predict  veracity
-1       False     1          1027
                   3           159
         True     -1           424
 0       False     1            37
         True     -1             8
                              ... 
 132     True     -1             4
 133     False     1            27
         True     -1            16
 134     False     1            10
 135     False     1            10
Name: count, Length: 250, dtype: int64
breaking further...
135
cluster  predict  veracity
-1       False     1          1035
                   3           160
         True     -1           428
 0       False     1            37
         True     -1             8
                              ... 
 132     True     -1             4
 133     False     1            27
         True     -1            16
 134     False     1            10
 135     False     1    

In [4]:
results_df = pd.json_normalize(results)
results_df

,percentage,score,accuracy,was_supervised_umap_used,metrics.accuracy,metrics.accuracy_not_including_fours,metrics.percentage_of_fours,metrics.percentage_of_no_clusters_in_ground_truth,metrics.precision_on_three,metrics.recall_on_three,...,metrics.recall_on_three_excluding_fours,metrics.accuracy_90_confidence,metrics.accuracy_80_confidence,metrics.accuracy_70_confidence,metrics.accuracy_60_confidence,metrics.percentage_90_confidence,metrics.percentage_80_confidence,metrics.percentage_70_confidence,metrics.percentage_60_confidence,metrics.cluster_df
0,1,1.810182,0.504435,True,0.504435,0.982721,0.486696,0.0,0.972973,0.290323,...,0.837209,0.402439,0.402439,0.402439,0.402439,0.987805,0.987805,0.987805,0.987805,...
